In [324]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter

In [325]:
data = pd.read_csv('movie_bd_v5.csv')
data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
203,tt2980706,50000000,151165787,Planes: Fire & Rescue,Dane Cook|Ed Harris|Julie Bowen|Curtis Armstro...,Roberts Gannaway,"When others fly out, heroes fly in.",When world-famous air racer Dusty learns that ...,83,Animation|Comedy|Adventure|Family,Walt Disney Pictures|DisneyToon Studios|Prana ...,7/17/2014,5.9,2014
411,tt1386588,100000000,170432927,The Other Guys,Will Ferrell|Mark Wahlberg|Eva Mendes|Samuel L...,Adam McKay,When the cops are busy... Our only hope is...,NYPD detectives Christopher Danson (Johnson) a...,107,Action|Comedy|Crime,Columbia Pictures|Gary Sanchez Productions,8/6/2010,6.1,2010
1426,tt0402901,30000000,15007991,The Cave,Cole Hauser|Eddie Cibrian|Morris Chestnut|Lena...,Bruce Hunt,There are places man was never meant to go.,After a group of biologists discovers a huge n...,97,Action|Adventure|Horror|Thriller,Lakeshore Entertainment|Cinerenta Medienbeteil...,8/25/2005,5.2,2005
900,tt0257044,80000000,181001478,Road to Perdition,Tom Hanks|Tyler Hoechlin|Jennifer Jason Leigh|...,Sam Mendes,Pray for Michael Sullivan.,Mike Sullivan works as a hit man for crime bos...,117,Thriller|Crime|Drama,DreamWorks SKG|The Zanuck Company|Twentieth Ce...,7/12/2002,7.1,2002
1711,tt0389722,30000000,75505973,30 Days of Night,Josh Hartnett|Melissa George|Ben Foster|Danny ...,David Slade,They're Coming!,This is the story of an isolated Alaskan town ...,113,Horror|Thriller,Columbia Pictures|Dark Horse Entertainment|Gho...,10/17/2007,6.1,2007


In [326]:
data.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


# Предобработка


In [327]:
'''постобработка для оригинального датасета'''
# вычисляем прибыль для каждой записи
data['profit'] = data['revenue'] - data['budget']

# выносим месяц выхода фильма в отдельное поле
dict_month = {
    '1': 'Jan',
    '2': 'Feb',
    '3': 'Mar',
    '4': 'Apr',
    '5': 'May',
    '6': 'Jun',
    '7': 'Jul',
    '8': 'Aug',
    '9': 'Sep',
    '10': 'Oct',
    '11': 'Nov',
    '12': 'Dec'
}
data['release_month'] = data['release_date'].apply(lambda x: dict_month.get(x.split('/')[0])) # выносим месяц выхода фильма в отдельное поле

# добавлям поля для количества букв в названии и слов в описании
data['original_title_len'] = data['original_title'].apply(lambda x: len(x))
data['overview_words_count'] = data['overview'].apply(lambda x: len(str(x).split(' ')))


'''создаем копию оригинального датасета с разгруппированными данными по жанру'''
data_split_genres = data.copy()
data_split_genres['genres'] = data_split_genres['genres'].str.split('|')
data_split_genres = data_split_genres.explode('genres')

result_genre = pd.DataFrame() # группируем новый датасет по другим категориям
result_genre['count_total'] = data_split_genres.groupby('genres').imdb_id.count()
result_genre['count_profitable'] = data_split_genres.query('profit > 0').groupby('genres').imdb_id.count()


'''создаем копию оригинального датасета с разгруппированными данными по жанру и режиссеру'''
data_split_genres_directors = data.copy()
data_split_genres_directors['genres'] = data_split_genres_directors['genres'].str.split('|')
data_split_genres_directors['director'] = data_split_genres_directors['director'].str.split('|')
data_split_genres_directors = data_split_genres_directors.explode('genres')
data_split_genres_directors = data_split_genres_directors.explode('director')

'''создаем копию оригинального датасета с разгруппированными данными по актерам'''
data_split_cast = data.copy()
data_split_cast['cast'] = data_split_cast['cast'].str.split('|')
data_split_cast = data_split_cast.explode('cast')

'''создаем копию оригинального датасета с разгруппированными данными по студиям'''
data_split_production_companies = data.copy()
data_split_production_companies['production_companies'] = data_split_production_companies['production_companies']\
    .str.split('|')
data_split_production_companies = data_split_production_companies.explode('production_companies')

'''исходная постобработка'''
answers = {} # создадим словарь для ответов

# тут другие ваши предобработки колонок например:

#the time given in the dataset is in string format.
#So we need to change this in datetime format
# ...
data.iloc[0][['overview', 'overview_words_count']]
#len(data.iloc[0]['overview'].split(' '))

overview                Twenty-two years after the events of Jurassic ...
overview_words_count                                                   26
Name: 0, dtype: object

# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

In [328]:
# в словарь вставляем номер вопроса и ваш ответ на него
# Пример: 
answers['1'] = '2. Spider-Man 3 (tt0413300)'
# запишите свой вариант ответа
answers['1'] = '5. Pirates of the Caribbean: On Stranger Tides (tt1298650)'
# если ответили верно, можете добавить комментарий со значком "+"

In [329]:
# тут пишем ваш код для решения данного вопроса:
data[data['budget'] == data['budget'].max()][['original_title', 'imdb_id']]

,original_title,imdb_id
723,Pirates of the Caribbean: On Stranger Tides,tt1298650


ВАРИАНТ 2

In [330]:
data.query('budget == budget.max()')[['original_title', 'imdb_id']]

,original_title,imdb_id
723,Pirates of the Caribbean: On Stranger Tides,tt1298650


# 2. Какой из фильмов самый длительный (в минутах)?

In [331]:
# думаю логику работы с этим словарем вы уже поняли, 
# по этому не буду больше его дублировать
answers['2'] = '2. Gods and Generals (tt0279111)'

In [332]:
data[data['runtime'] == data['runtime'].max()][['original_title', 'imdb_id']]


,original_title,imdb_id
1157,Gods and Generals,tt0279111


# 3. Какой из фильмов самый короткий (в минутах)?


In [333]:
answers['3'] = '3. Winnie the Pooh (tt1449283)'

data[data['runtime'] == data['runtime'].min()][['original_title', 'imdb_id']]


,original_title,imdb_id
768,Winnie the Pooh,tt1449283


# 4. Какова средняя длительность фильмов?


In [334]:
answers['4'] = '2. 110'

data.runtime.mean().round()

110.0

# 5. Каково медианное значение длительности фильмов? 

In [335]:
answers['5'] = '1. 107'

data.runtime.median().round()

107.0

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [336]:
answers['6'] = '5. Avatar (tt0499549)'

data[data['profit'] == data['profit'].max()][['original_title', 'imdb_id']]

# лучше код получения столбца profit вынести в Предобработку что в начале

,original_title,imdb_id
239,Avatar,tt0499549


# 7. Какой фильм самый убыточный? 

In [337]:
answers['7'] = '5. The Lone Ranger	 (tt1210819)'

data[data['profit'] == data['profit'].min()][['original_title', 'imdb_id']]

,original_title,imdb_id
1245,The Lone Ranger,tt1210819


# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [338]:
answers['8'] = '1. 1478'

data[data['revenue'] > data['budget']]['imdb_id'].count()


1478

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [339]:
answers['9'] = '4. The Dark Knight	(tt0468569)'

data.query('release_year == 2008').query('revenue == revenue.max()')[['original_title', 'imdb_id']]

,original_title,imdb_id
599,The Dark Knight,tt0468569


# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [340]:
answers['10'] = '5. The Lone Ranger	(tt1210819)'

data.query('release_year in [20012, 2013, 2014]').query('profit == profit.min()')[['original_title', 'imdb_id']]


,original_title,imdb_id
1245,The Lone Ranger,tt1210819


# 11. Какого жанра фильмов больше всего?

In [341]:
answers['11'] = '3. Drama'

result_genre.query('count_total == count_total.max()') # группировка проделана в постобработке



# эту задачу тоже можно решать разными подходами, попробуй реализовать разные варианты
# если будешь добавлять функцию - выноси ее в предобработку что в начале

,count_total,count_profitable
genres,,
Drama,782,560.0


ВАРИАНТ 2

# 12. Фильмы какого жанра чаще всего становятся прибыльными?



In [342]:
answers['12'] = '1. Drama'

result_genre.query('count_profitable == count_profitable.max()') # группировка проделана в постобработке

,count_total,count_profitable
genres,,
Drama,782,560.0


# 13. У какого режиссера самые большие суммарные кассовые сбооры?

In [343]:
answers['13'] = '5. Peter Jackson'

data.groupby('director')['revenue'].sum().sort_values(ascending=False)

director
Peter Jackson        6490593685
Christopher Nolan    4167548502
David Yates          4154295625
Michael Bay          3886938960
J.J. Abrams          3579169916
                        ...    
David MichÃ´d           2295423
Steven Shainberg        2281089
Paul Schrader           2062066
Keanu Reeves            2054941
Simon Hunter            2033165
Name: revenue, Length: 957, dtype: int64

# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [344]:
answers['14'] = '3. Robert Rodriguez'

#len(data_split_genres_directors['imdb_id'].unique()) == len(data['imdb_id'].unique())
data_split_genres_directors.query('genres == "Action"').groupby('director')['imdb_id'].count().sort_values(ascending=False)



director
Robert Rodriguez      9
Paul W.S. Anderson    7
Michael Bay           7
Ridley Scott          6
Antoine Fuqua         6
                     ..
Joe Cornish           1
Roger Spottiswoode    1
Jimmy Hayward         1
Jim Gillespie         1
Jon Avnet             1
Name: imdb_id, Length: 364, dtype: int64

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [345]:
answers['15'] = '3. Chris Hemsworth'

data_split_cast.query('release_year == 2012').groupby('cast').revenue.sum().sort_values(ascending=False)

cast
Chris Hemsworth      2027450773
Denis Leary          1629460639
Anne Hathaway        1522851057
Chris Evans          1519557910
Robert Downey Jr.    1519557910
                        ...    
Jason Bateman           3428048
Danny Huston            2106557
Sami Gayle              2106557
Josh Lucas              2106557
Nicolas Cage            2106557
Name: revenue, Length: 466, dtype: int64

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [346]:
answers['16'] = '3. Matt Damon'

data_split_cast.query('budget > budget.mean()').groupby('cast')['imdb_id'].count().sort_values(ascending=False)

cast
Matt Damon           18
Adam Sandler         17
Angelina Jolie       16
Eddie Murphy         15
Samuel L. Jackson    15
                     ..
Leslie Bibb           1
Leonard Nimoy         1
Lennie James          1
Lena Olin             1
50 Cent               1
Name: imdb_id, Length: 1505, dtype: int64

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [347]:
answers['17'] = '2. Action'

data_split_genres[data_split_genres.cast.str.contains('Nicolas Cage')].groupby('genres')['imdb_id']\
    .count().sort_values(ascending=False)


genres
Action             17
Thriller           15
Drama              12
Crime              10
Fantasy             8
Adventure           7
Comedy              6
Science Fiction     4
Mystery             3
Family              3
Animation           3
History             2
War                 1
Romance             1
Horror              1
Name: imdb_id, dtype: int64

# 18. Самый убыточный фильм от Paramount Pictures

In [348]:
answers['18'] = '1. K-19: The Widowmaker (tt0267626)'

data[data['production_companies'].str.contains('Paramount Pictures')]\
    .query('profit == profit.min()')[['profit','original_title','imdb_id']]

,profit,original_title,imdb_id
925,-64831034,K-19: The Widowmaker,tt0267626


# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [349]:
answers['19'] = '5. 2015'
data.groupby('release_year')['revenue'].sum().sort_values(ascending=False)

release_year
2015    25449202382
2014    23405862953
2013    23213799791
2012    23079001687
2011    22676791872
2010    21071204961
2009    20261791024
2008    18252781990
2007    18162406801
2004    15663430720
2005    15309425558
2006    14775042320
2003    14346123312
2002    14136361487
2001    13017764865
2000    10664099805
Name: revenue, dtype: int64

# 20. Какой самый прибыльный год для студии Warner Bros?

In [350]:
answers['20'] = '1. 2014'

data[data['production_companies'].str.contains('Warner Bros')]\
    .groupby('release_year')['profit'].sum().sort_values(ascending=False)

release_year
2014    2295464519
2007    2201675217
2008    2134595031
2010    1974712985
2011    1871393682
2003    1855493377
2009    1822454136
2013    1636453400
2004    1631933725
2005    1551980298
2001    1343545668
2012    1258020056
2002    1022709901
2015     870368348
2006     620170743
2000     452631386
Name: profit, dtype: int64

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [351]:
answers['21'] = '4. Сентябрь'
data.groupby('release_month')['imdb_id'].count().sort_values(ascending=False)


release_month
Sep    227
Dec    190
Oct    186
Aug    161
Mar    156
Apr    149
Jun    147
Nov    146
Jul    142
May    140
Feb    135
Jan    110
Name: imdb_id, dtype: int64

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [352]:
answers['22'] = '2. 450'

list_summer = ['Jun', 'Jul', 'Aug']
data.query('release_month in @list_summer')['imdb_id'].count()

450

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [353]:
answers['23'] = '3. Peter Jackson'

list_winter = ['Dec', 'Jan', 'Feb']
data_split_genres_directors.query('release_month in @list_winter')\
    .groupby('director')['imdb_id'].count().sort_values(ascending=False)

director
Peter Jackson         18
Steven Soderbergh     14
Clint Eastwood        13
Rob Marshall          11
Adam Shankman         11
                      ..
Bill Condon            1
Jason Moore            1
David Gordon Green     1
Rob Bowman             1
Jack Sholder           1
Name: imdb_id, Length: 358, dtype: int64

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?



In [354]:
answers['24'] = '5. Four By Two Productions'
data_split_production_companies.groupby('production_companies')['original_title_len'].max().sort_values(ascending=False)

production_companies
Four By Two Productions                   83
Twentieth Century Fox Film Corporation    83
Walden Media                              62
Walt Disney                               62
Jim Henson Company, The                   59
                                          ..
Everest Entertainment                      3
Berlanti Productions                       3
XM2 Productions                            2
Global Entertainment Group                 2
Ixtlan Productions                         2
Name: original_title_len, Length: 1771, dtype: int64

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [355]:
answers['25'] = '3. Midnight Picture Show'

data_split_production_companies.groupby('production_companies')['overview_words_count']\
    .mean().sort_values(ascending=False)

production_companies
Midnight Picture Show                    175.0
Room 9 Entertainment                     161.0
Heineken Branded Entertainment           161.0
98 MPH Productions                       159.0
Brookwell-McNamara Entertainment         156.0
                                         ...  
London Boulevard                          13.0
Phantom Four                              13.0
Henceforth                                13.0
Empire Pictures                           11.0
Motion Picture Corporation of America     11.0
Name: overview_words_count, Length: 1771, dtype: float64

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [356]:
answers['26'] = '1. Inside Out, The Dark Knight, 12 Years a Slave'

data[data.vote_average >= data.vote_average.quantile(.99)][['original_title', 'vote_average']]\
    .sort_values('vote_average', ascending=False)

,original_title,vote_average
599,The Dark Knight,8.1
9,Inside Out,8.0
34,Room,8.0
118,Interstellar,8.0
125,The Imitation Game,8.0
370,Inception,7.9
1191,12 Years a Slave,7.9
1183,The Wolf of Wall Street,7.9
1081,The Lord of the Rings: The Return of the King,7.9
872,The Pianist,7.9


# 27. Какие актеры чаще всего снимаются в одном фильме вместе?

In [357]:
answers['27'] = '5. Daniel Radcliffe & Rupert Grint'

from itertools import combinations
from collections import Counter
pairs = []
for group in [x.split('|') for x in data.cast]:
    pairs.extend(list(combinations(group, 2)))
counted_pairs = Counter(pairs)
result = counted_pairs.most_common()
result[0]


(('Daniel Radcliffe', 'Rupert Grint'), 8)

# Submission

In [358]:
# в конце можно посмотреть свои ответы к каждому вопросу
answers

{'1': '5. Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': '2. Gods and Generals (tt0279111)',
 '3': '3. Winnie the Pooh (tt1449283)',
 '4': '2. 110',
 '5': '1. 107',
 '6': '5. Avatar (tt0499549)',
 '7': '5. The Lone Ranger\t (tt1210819)',
 '8': '1. 1478',
 '9': '4. The Dark Knight\t(tt0468569)',
 '10': '5. The Lone Ranger\t(tt1210819)',
 '11': '3. Drama',
 '12': '1. Drama',
 '13': '5. Peter Jackson',
 '14': '3. Robert Rodriguez',
 '15': '3. Chris Hemsworth',
 '16': '3. Matt Damon',
 '17': '2. Action',
 '18': '1. K-19: The Widowmaker (tt0267626)',
 '19': '5. 2015',
 '20': '1. 2014',
 '21': '4. Сентябрь',
 '22': '2. 450',
 '23': '3. Clint Eastwood',
 '24': '5. Four By Two Productions',
 '25': '3. Midnight Picture Show',
 '26': '1. Inside Out, The Dark Knight, 12 Years a Slave',
 '27': '5. Daniel Radcliffe & Rupert Grint'}

In [359]:
# и убедиться что ни чего не пропустил)
len(answers)

27